In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.regression.linear_model import OLS
from itertools import combinations
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [7]:
def find_cointegrated_pairs(tickers, 
                           correlation_threshold=0.7,
                           adf_pvalue_threshold=0.05,
                           lookback_years=10,
                           individual_adf_pvalue=0.05):
    """
    Finds cointegrated pairs among a list of stock tickers.
    
    Cointegration Requirements (Engle-Granger):
    - Each series is individually I(1) - non-stationary in levels
    - Linear combination (spread/residuals) is I(0) - stationary
    - ADF test on residuals from OLS regression
    - Johansen test for validation
    
    Parameters:
    -----------
    tickers : list
        List of ticker symbols (20+)
    correlation_threshold : float
        Minimum correlation coefficient (default: 0.7)
    adf_pvalue_threshold : float
        Maximum p-value for ADF test on residuals (default: 0.05)
    lookback_years : int
        Years of historical data to analyze (default: 10)
    individual_adf_pvalue : float
        Maximum p-value for individual series to be I(1) (default: 0.05)
        Series should FAIL to reject null (p > threshold) to be non-stationary
    
    Returns:
    --------
    list of tuples
        Cointegrated pairs: [(ticker1, ticker2, stats_dict), ...]
    """
    
    print(f"Fetching {lookback_years} years of data for {len(tickers)} tickers...")
    
    # Download historical data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=lookback_years*365)
    
    data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Close']
    
    # Handle single ticker case (returns Series instead of DataFrame)
    if isinstance(data, pd.Series):
        data = data.to_frame()
    
    # Drop tickers with insufficient data
    min_data_points = int(lookback_years * 252 * 0.8)  # 80% of expected trading days
    data = data.dropna(axis=1, thresh=min_data_points)
    
    available_tickers = data.columns.tolist()
    print(f"Valid tickers with sufficient data: {len(available_tickers)}")
    
    if len(available_tickers) < 2:
        print("Insufficient tickers with valid data.")
        return []
    
    # Drop rows with any NaN values for pairwise analysis
    data = data.dropna()
    
    print(f"Data shape: {data.shape} (rows: {data.shape[0]}, tickers: {data.shape[1]})")
    
    print("\nStep 1: Testing individual series for non-stationarity I(1)...")
    
    # Test each series individually - they should be I(1) (non-stationary in levels)
    stationary_tickers = []
    non_stationary_tickers = []
    
    for ticker in available_tickers:
        prices = data[ticker].values
        adf_result = adfuller(prices, maxlag=1, regression='c')
        adf_pvalue = adf_result[1]
        
        # For I(1): we want to FAIL to reject null hypothesis (non-stationary)
        # So p-value should be > threshold (e.g., > 0.05)
        if adf_pvalue > individual_adf_pvalue:
            non_stationary_tickers.append(ticker)
            print(f"  ✓ {ticker}: Non-stationary (p={adf_pvalue:.4f}) - I(1)")
        else:
            stationary_tickers.append(ticker)
            print(f"  ✗ {ticker}: Stationary (p={adf_pvalue:.4f}) - I(0), excluding")
    
    print(f"\nValid I(1) tickers: {len(non_stationary_tickers)}")
    
    if len(non_stationary_tickers) < 2:
        print("Insufficient non-stationary tickers for cointegration analysis.")
        return []
    
    # Filter data to only include non-stationary tickers
    data = data[non_stationary_tickers]
    
    print("\nStep 2: Calculating correlation matrix...")
    
    # Calculate correlation matrix
    correlation_matrix = data.corr()
    
    # Find pairs with high correlation (only among non-stationary tickers)
    highly_correlated_pairs = []
    for i, ticker1 in enumerate(non_stationary_tickers):
        for ticker2 in non_stationary_tickers[i+1:]:
            corr = correlation_matrix.loc[ticker1, ticker2]
            if corr > correlation_threshold:
                highly_correlated_pairs.append((ticker1, ticker2, corr))
    
    print(f"Found {len(highly_correlated_pairs)} pairs with correlation > {correlation_threshold}")
    
    if len(highly_correlated_pairs) == 0:
        print("No highly correlated pairs found.")
        return []
    
    print("\nStep 3: Testing for cointegration (Engle-Granger)...")
    cointegrated_pairs = []
    
    for ticker1, ticker2, corr in highly_correlated_pairs:
        try:
            prices1 = data[ticker1].values
            prices2 = data[ticker2].values
            
            # Engle-Granger two-step procedure:
            # Step 1: Run OLS regression on price levels: Y = α + β*X + ε
            X = np.column_stack([np.ones(len(prices2)), prices2])
            model = OLS(prices1, X).fit()
            residuals = model.resid  # This is epsilon (the spread/error)
            hedge_ratio = model.params[1]  # Beta coefficient
            
            # Step 2: Test if residuals (epsilon/spread) are stationary I(0)
            # Apply Augmented Dickey-Fuller test to residuals
            adf_result = adfuller(residuals, maxlag=1, regression='c')
            adf_statistic = adf_result[0]
            adf_pvalue = adf_result[1]
            
            # For cointegration: residuals must be stationary
            # We want to REJECT null hypothesis (p-value < threshold)
            if adf_pvalue < adf_pvalue_threshold:
                # Perform Johansen test for additional validation
                # Johansen test requires 2D array of both price series
                price_matrix = np.column_stack([prices1, prices2])
                
                try:
                    johansen_result = coint_johansen(price_matrix, det_order=0, k_ar_diff=1)
                    # Check trace statistic against 95% critical value
                    trace_stat = johansen_result.lr1[0]  # First eigenvalue trace stat
                    critical_value_95 = johansen_result.cvt[0, 1]  # 95% critical value
                    johansen_pass = trace_stat > critical_value_95
                    
                    if johansen_pass:
                        stats = {
                            'correlation': round(corr, 4),
                            'hedge_ratio': round(hedge_ratio, 4),
                            'adf_statistic': round(adf_statistic, 4),
                            'adf_pvalue': round(adf_pvalue, 6),
                            'johansen_trace': round(trace_stat, 4),
                            'johansen_critical_95': round(critical_value_95, 4),
                            'spread_mean': round(np.mean(residuals), 4),
                            'spread_std': round(np.std(residuals), 4)
                        }
                        cointegrated_pairs.append((ticker1, ticker2, stats))
                        print(f"✓ {ticker1} - {ticker2}: COINTEGRATED")
                        print(f"    Correlation: {corr:.4f} | Hedge Ratio: {hedge_ratio:.4f}")
                        print(f"    ADF p-value: {adf_pvalue:.6f} (residuals stationary)")
                        print(f"    Johansen: {trace_stat:.2f} > {critical_value_95:.2f}")
                    else:
                        print(f"✗ {ticker1} - {ticker2}: Failed Johansen test ({trace_stat:.2f} < {critical_value_95:.2f})")
                    
                except Exception as e:
                    # If Johansen test fails, skip this pair
                    print(f"✗ {ticker1} - {ticker2}: Johansen test failed - {str(e)}")
                    continue
            
        except Exception as e:
            print(f"✗ {ticker1} - {ticker2}: Error during testing - {str(e)}")
            continue
    
    print(f"\n{'='*60}")
    print(f"RESULTS: Found {len(cointegrated_pairs)} cointegrated pairs")
    print(f"{'='*60}")
    
    return cointegrated_pairs

In [14]:
tickers = [
        'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'JPM',
        'BAC', 'WFC', 'C', 'GS', 'MS', 'XOM', 'CVX', 'COP', 'KO', 'PEP',
        'WMT', 'TGT', 'HD', 'LOW', 'NKE', 'SBUX', 'MCD'
    ]

# Comprehensive economic sector classification
SECTOR_CLASSIFICATION = {
    # Technology
    'AAPL': 'Tech - Consumer Electronics',
    'MSFT': 'Tech - Software & Cloud',
    'GOOGL': 'Tech - Internet & Advertising',
    'GOOG': 'Tech - Internet & Advertising',
    'META': 'Tech - Social Media & Advertising',
    'AMZN': 'Tech - E-commerce & Cloud',
    'NVDA': 'Tech - Semiconductors',
    'AMD': 'Tech - Semiconductors',
    'INTC': 'Tech - Semiconductors',
    'TSM': 'Tech - Semiconductors',
    'QCOM': 'Tech - Semiconductors',
    'AVGO': 'Tech - Semiconductors',
    'CRM': 'Tech - Software & Cloud',
    'ORCL': 'Tech - Software & Cloud',
    'ADBE': 'Tech - Software & Cloud',
    'CSCO': 'Tech - Networking Equipment',
    'IBM': 'Tech - IT Services',
    'SNOW': 'Tech - Software & Cloud',
    
    # Financials - Banks
    'JPM': 'Financial - Diversified Banks',
    'BAC': 'Financial - Diversified Banks',
    'WFC': 'Financial - Diversified Banks',
    'C': 'Financial - Diversified Banks',
    'USB': 'Financial - Regional Banks',
    'PNC': 'Financial - Regional Banks',
    'TFC': 'Financial - Regional Banks',
    
    # Financials - Investment Banking
    'GS': 'Financial - Investment Banking',
    'MS': 'Financial - Investment Banking',
    
    # Financials - Insurance
    'BRK.B': 'Financial - Insurance',
    'PGR': 'Financial - Insurance',
    'AIG': 'Financial - Insurance',
    'MET': 'Financial - Insurance',
    
    # Financials - Asset Management
    'BLK': 'Financial - Asset Management',
    'SCHW': 'Financial - Asset Management',
    
    # Financials - Payment Processing
    'V': 'Financial - Payment Processing',
    'MA': 'Financial - Payment Processing',
    'PYPL': 'Financial - Payment Processing',
    
    # Energy - Oil & Gas Integrated
    'XOM': 'Energy - Oil & Gas Integrated',
    'CVX': 'Energy - Oil & Gas Integrated',
    'BP': 'Energy - Oil & Gas Integrated',
    'SHEL': 'Energy - Oil & Gas Integrated',
    'TTE': 'Energy - Oil & Gas Integrated',
    
    # Energy - Oil & Gas E&P
    'COP': 'Energy - Oil & Gas Exploration',
    'EOG': 'Energy - Oil & Gas Exploration',
    'PXD': 'Energy - Oil & Gas Exploration',
    
    # Energy - Oil Services
    'SLB': 'Energy - Oil Services',
    'HAL': 'Energy - Oil Services',
    
    # Consumer Discretionary - Retail
    'WMT': 'Consumer - Discount Retail',
    'COST': 'Consumer - Warehouse Retail',
    'TGT': 'Consumer - Discount Retail',
    'HD': 'Consumer - Home Improvement',
    'LOW': 'Consumer - Home Improvement',
    
    # Consumer Discretionary - Automotive
    'TSLA': 'Consumer - Electric Vehicles',
    'F': 'Consumer - Traditional Automotive',
    'GM': 'Consumer - Traditional Automotive',
    'TM': 'Consumer - Traditional Automotive',
    
    # Consumer Discretionary - Restaurants
    'MCD': 'Consumer - Quick Service Restaurants',
    'SBUX': 'Consumer - Coffee & Cafes',
    'YUM': 'Consumer - Quick Service Restaurants',
    'CMG': 'Consumer - Fast Casual Restaurants',
    
    # Consumer Discretionary - Apparel
    'NKE': 'Consumer - Athletic Apparel',
    'LULU': 'Consumer - Athletic Apparel',
    'ADDYY': 'Consumer - Athletic Apparel',
    
    # Consumer Discretionary - E-commerce
    'BABA': 'Consumer - E-commerce China',
    'JD': 'Consumer - E-commerce China',
    
    # Consumer Staples - Beverages
    'KO': 'Consumer Staples - Soft Drinks',
    'PEP': 'Consumer Staples - Soft Drinks & Snacks',
    'MNST': 'Consumer Staples - Energy Drinks',
    
    # Consumer Staples - Food
    'PG': 'Consumer Staples - Household Products',
    'KMB': 'Consumer Staples - Household Products',
    'CL': 'Consumer Staples - Household Products',
    'GIS': 'Consumer Staples - Packaged Foods',
    'K': 'Consumer Staples - Packaged Foods',
    
    # Healthcare - Pharmaceuticals
    'JNJ': 'Healthcare - Pharmaceuticals',
    'PFE': 'Healthcare - Pharmaceuticals',
    'MRK': 'Healthcare - Pharmaceuticals',
    'ABBV': 'Healthcare - Pharmaceuticals',
    'LLY': 'Healthcare - Pharmaceuticals',
    'NVO': 'Healthcare - Pharmaceuticals',
    
    # Healthcare - Biotechnology
    'AMGN': 'Healthcare - Biotechnology',
    'GILD': 'Healthcare - Biotechnology',
    'BIIB': 'Healthcare - Biotechnology',
    'VRTX': 'Healthcare - Biotechnology',
    
    # Healthcare - Medical Devices
    'MDT': 'Healthcare - Medical Devices',
    'ABT': 'Healthcare - Medical Devices',
    'TMO': 'Healthcare - Medical Devices',
    
    # Healthcare - Health Insurance
    'UNH': 'Healthcare - Health Insurance',
    'CVS': 'Healthcare - Pharmacy & Insurance',
    'CI': 'Healthcare - Health Insurance',
    
    # Industrials - Aerospace
    'BA': 'Industrials - Aerospace',
    'LMT': 'Industrials - Aerospace & Defense',
    'RTX': 'Industrials - Aerospace & Defense',
    'GD': 'Industrials - Aerospace & Defense',
    
    # Industrials - Transportation
    'UPS': 'Industrials - Package Delivery',
    'FDX': 'Industrials - Package Delivery',
    'DAL': 'Industrials - Airlines',
    'UAL': 'Industrials - Airlines',
    'AAL': 'Industrials - Airlines',
    
    # Industrials - Manufacturing
    'CAT': 'Industrials - Heavy Equipment',
    'DE': 'Industrials - Agricultural Equipment',
    
    # Telecommunications
    'T': 'Telecom - Wireless & Media',
    'VZ': 'Telecom - Wireless',
    'TMUS': 'Telecom - Wireless',
    
    # Utilities
    'NEE': 'Utilities - Electric',
    'DUK': 'Utilities - Electric',
    'SO': 'Utilities - Electric',
    'D': 'Utilities - Electric',
    
    # Real Estate
    'AMT': 'Real Estate - Cell Towers',
    'PLD': 'Real Estate - Industrial',
    'SPG': 'Real Estate - Retail Malls',
    'O': 'Real Estate - Retail REIT',
    
    # Materials - Chemicals
    'LIN': 'Materials - Industrial Gases',
    'APD': 'Materials - Industrial Gases',
    'DOW': 'Materials - Chemicals',
    
    # Materials - Mining
    'FCX': 'Materials - Copper Mining',
    'NEM': 'Materials - Gold Mining',
    'GOLD': 'Materials - Gold Mining',
}


In [8]:
pairs = find_cointegrated_pairs(
        tickers, 
        correlation_threshold=0.7,
        adf_pvalue_threshold=0.05,
        lookback_years=10
    )

Fetching 10 years of data for 25 tickers...
Valid tickers with sufficient data: 25
Data shape: (2512, 25) (rows: 2512, tickers: 25)

Step 1: Testing individual series for non-stationarity I(1)...
  ✓ AAPL: Non-stationary (p=0.9760) - I(1)
  ✓ AMZN: Non-stationary (p=0.9112) - I(1)
  ✓ BAC: Non-stationary (p=0.8886) - I(1)
  ✓ C: Non-stationary (p=0.9413) - I(1)
  ✓ COP: Non-stationary (p=0.6693) - I(1)
  ✓ CVX: Non-stationary (p=0.6467) - I(1)
  ✓ GOOGL: Non-stationary (p=0.9988) - I(1)
  ✓ GS: Non-stationary (p=0.9984) - I(1)
  ✓ HD: Non-stationary (p=0.7781) - I(1)
  ✓ JPM: Non-stationary (p=0.9975) - I(1)
  ✓ KO: Non-stationary (p=0.8061) - I(1)
  ✓ LOW: Non-stationary (p=0.7801) - I(1)
  ✓ MCD: Non-stationary (p=0.7642) - I(1)
  ✓ META: Non-stationary (p=0.9496) - I(1)
  ✓ MS: Non-stationary (p=0.9955) - I(1)
  ✓ MSFT: Non-stationary (p=0.9861) - I(1)
  ✓ NKE: Non-stationary (p=0.5148) - I(1)
  ✓ NVDA: Non-stationary (p=1.0000) - I(1)
  ✓ PEP: Non-stationary (p=0.5376) - I(1)
  ✓ S

In [15]:
def filter_economically_related_pairs(pairs, sector_classification=None):
    """
    Filters cointegrated pairs to keep only those with economic relationships.
    
    Economic relationship criteria:
    - Same sub-sector (strongest relationship)
    - Same broad sector with related business models
    - Supply chain relationships
    - Competitive dynamics
    
    Parameters:
    -----------
    pairs : list
        List of tuples from find_cointegrated_pairs()
        Format: [(ticker1, ticker2, stats_dict), ...]
    sector_classification : dict, optional
        Custom sector mapping. If None, uses default SECTOR_CLASSIFICATION
    
    Returns:
    --------
    list of tuples
        Filtered pairs with economic relationships
        Format: [(ticker1, ticker2, stats_dict, relationship_type), ...]
    """
    
    if sector_classification is None:
        sector_classification = SECTOR_CLASSIFICATION
    
    economically_related_pairs = []
    
    print(f"\nFiltering {len(pairs)} cointegrated pairs for economic relationships...")
    print("=" * 80)
    
    for ticker1, ticker2, stats in pairs:
        # Get sectors for both tickers
        sector1 = sector_classification.get(ticker1, 'Unknown')
        sector2 = sector_classification.get(ticker2, 'Unknown')
        
        # Skip if either ticker not classified
        if sector1 == 'Unknown' or sector2 == 'Unknown':
            print(f"✗ {ticker1} - {ticker2}: Unclassified ticker(s)")
            print(f"  {ticker1}: {sector1}")
            print(f"  {ticker2}: {sector2}")
            continue
        
        # Extract broad sector and sub-sector
        broad1 = sector1.split(' - ')[0]
        broad2 = sector2.split(' - ')[0]
        
        relationship_type = None
        
        # Check for exact sub-sector match (strongest relationship)
        if sector1 == sector2:
            relationship_type = "Same Sub-Sector"
            economically_related_pairs.append((ticker1, ticker2, stats, relationship_type))
            print(f"✓ {ticker1} - {ticker2}: {relationship_type}")
            print(f"  Sector: {sector1}")
            print(f"  Correlation: {stats['correlation']:.4f} | Hedge Ratio: {stats['hedge_ratio']:.4f}")
            
        # Check for same broad sector (related business models)
        elif broad1 == broad2:
            relationship_type = f"Same Broad Sector ({broad1})"
            economically_related_pairs.append((ticker1, ticker2, stats, relationship_type))
            print(f"✓ {ticker1} - {ticker2}: {relationship_type}")
            print(f"  {ticker1}: {sector1}")
            print(f"  {ticker2}: {sector2}")
            print(f"  Correlation: {stats['correlation']:.4f} | Hedge Ratio: {stats['hedge_ratio']:.4f}")
            
        # Check for cross-sector economic relationships
        else:
            # Energy companies and energy-intensive industries
            energy_intensive = ['Industrials - Airlines', 'Consumer - Traditional Automotive']
            energy_producers = ['Energy - Oil & Gas Integrated', 'Energy - Oil & Gas Exploration']
            
            if (sector1 in energy_producers and sector2 in energy_intensive) or \
               (sector2 in energy_producers and sector1 in energy_intensive):
                relationship_type = "Supply Chain (Energy)"
                economically_related_pairs.append((ticker1, ticker2, stats, relationship_type))
                print(f"✓ {ticker1} - {ticker2}: {relationship_type}")
                print(f"  {ticker1}: {sector1}")
                print(f"  {ticker2}: {sector2}")
                
            # Retail and consumer goods
            elif (broad1 == 'Consumer' and broad2 == 'Consumer Staples') or \
                 (broad1 == 'Consumer Staples' and broad2 == 'Consumer'):
                relationship_type = "Related Consumer Sectors"
                economically_related_pairs.append((ticker1, ticker2, stats, relationship_type))
                print(f"✓ {ticker1} - {ticker2}: {relationship_type}")
                print(f"  {ticker1}: {sector1}")
                print(f"  {ticker2}: {sector2}")
                
            # Tech ecosystem relationships
            elif broad1 == 'Tech' and broad2 == 'Tech':
                relationship_type = "Tech Ecosystem"
                economically_related_pairs.append((ticker1, ticker2, stats, relationship_type))
                print(f"✓ {ticker1} - {ticker2}: {relationship_type}")
                print(f"  {ticker1}: {sector1}")
                print(f"  {ticker2}: {sector2}")
                
            else:
                print(f"✗ {ticker1} - {ticker2}: No clear economic relationship")
                print(f"  {ticker1}: {sector1}")
                print(f"  {ticker2}: {sector2}")
    
    print("\n" + "=" * 80)
    print(f"RESULTS: {len(economically_related_pairs)} pairs with economic relationships")
    print(f"Filtered out: {len(pairs) - len(economically_related_pairs)} spurious pairs")
    print("=" * 80)
    
    return economically_related_pairs

In [16]:
# Filter for economic relationships
if pairs:
    print("\n" + "="*80)
    print("FILTERING FOR ECONOMIC RELATIONSHIPS")
    print("="*80)
        
    economically_related = filter_economically_related_pairs(pairs)
        
    # Display final results
    if economically_related:
        print("\n" + "="*80)
        print("FINAL ECONOMICALLY RELATED COINTEGRATED PAIRS")
        print("="*80)
        for ticker1, ticker2, stats, relationship in economically_related:
            print(f"\n{ticker1} <-> {ticker2}")
            print(f"  Relationship:    {relationship}")
            print(f"  Correlation:     {stats['correlation']:.4f}")
            print(f"  Hedge Ratio:     {stats['hedge_ratio']:.4f}")
            print(f"  ADF p-value:     {stats['adf_pvalue']:.6f}")
            print(f"  Spread Mean:     {stats['spread_mean']:.4f}")
            print(f"  Spread Std:      {stats['spread_std']:.4f}")
    else:
        print("\nNo economically related cointegrated pairs found.")
else:
    print("\nNo cointegrated pairs found with the given criteria.")


FILTERING FOR ECONOMIC RELATIONSHIPS

Filtering 15 cointegrated pairs for economic relationships...
✗ AAPL - HD: No clear economic relationship
  AAPL: Tech - Consumer Electronics
  HD: Consumer - Home Improvement
✗ AAPL - KO: No clear economic relationship
  AAPL: Tech - Consumer Electronics
  KO: Consumer Staples - Soft Drinks
✗ AAPL - LOW: No clear economic relationship
  AAPL: Tech - Consumer Electronics
  LOW: Consumer - Home Improvement
✓ AAPL - MSFT: Same Broad Sector (Tech)
  AAPL: Tech - Consumer Electronics
  MSFT: Tech - Software & Cloud
  Correlation: 0.9770 | Hedge Ratio: 0.5066
✗ BAC - GOOGL: No clear economic relationship
  BAC: Financial - Diversified Banks
  GOOGL: Tech - Internet & Advertising
✗ GOOGL - GS: No clear economic relationship
  GOOGL: Tech - Internet & Advertising
  GS: Financial - Investment Banking
✗ GOOGL - MS: No clear economic relationship
  GOOGL: Tech - Internet & Advertising
  MS: Financial - Investment Banking
✗ GS - NVDA: No clear economic relat